# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [82]:
#Load the csv exported in Part I to a DataFrame
city_file_location = "Resources/city_weather_data (1).csv"
city_df = pd.read_csv(city_file_location)
city_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mattru,7.6244,-11.8332,26.13,64.0,100.0,2.80,SL,1.611082e+09
1,puerto ayora,-0.7393,-90.3518,28.00,74.0,20.0,7.72,EC,1.611082e+09
2,ushuaia,-54.8000,-68.3000,15.00,47.0,20.0,3.60,AR,1.611082e+09
3,chapais,49.7834,-74.8492,-18.00,71.0,75.0,0.53,CA,1.611082e+09
4,hilo,19.7297,-155.0900,20.00,88.0,90.0,5.66,US,1.611081e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Access maps with unique API key
gmaps.configure(g_key)

In [50]:

#Set parameters for gmap
locations = city_df[["Lat", "lng"]]
humidity = city_df["Humidity"].astype(float)

#Plot the heatmap
figure = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=50,point_radius=1.5)

#add the layer to the figure
figure.add_layer(heat_layer)
4

#display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
#create a new df to allow for weather criteria. Max Temp lower than 80 bu thigher than 70
#wind speed less than 10 mph, zero cloudiness

conditions_df = city_df.copy()
conditions_df = conditions_df.loc[conditions_df["Max Temp"] >=30]
# conditions_df = conditions_df.loc[conditions_df["Max Temp"] <=80]
# conditions_df = conditions_df.loc[conditions_df["Wind Speed"] <=10]
# conditions_df = conditions_df.loc[conditions_df["Cloudiness"] ==0]
conditions_df = conditions_df.dropna()
conditions_df = conditions_df.reset_index(drop=True)
conditions_df


# #drop any additional columns from your data set
# conditions_df_drop = conditions_df.drop(['Index', 'Unnamed: 0'], axis=1)
# conditions_df_drop

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lodwar,3.1191,35.5973,30.40,24.0,36.0,3.74,KE,1.611082e+09
1,tiarei,-17.5333,-149.3333,30.00,58.0,20.0,1.54,PF,1.611082e+09
2,sao joao da barra,-21.6403,-41.0511,31.00,51.0,0.0,9.26,BR,1.611081e+09
3,arraial do cabo,-22.9661,-42.0278,33.00,58.0,0.0,6.69,BR,1.611081e+09
4,tautira,-17.7333,-149.1500,30.00,58.0,20.0,1.54,PF,1.611082e+09
5,neuquen,-38.9516,-68.0591,35.00,23.0,0.0,3.09,AR,1.611082e+09
6,acapulco,16.8634,-99.8901,31.00,51.0,90.0,2.06,MX,1.611082e+09
7,panama city,8.9936,-79.5197,32.22,55.0,20.0,7.20,PA,1.611082e+09
8,linares,-35.8500,-71.6000,38.62,16.0,0.0,1.55,CL,1.611082e+09
9,gunjur,13.2019,-16.7339,32.00,29.0,69.0,7.97,GM,1.611082e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create a dataframe with the columns that will be used for the project / add an additonal column for the hotel names
hotel_df = pd.DataFrame(conditions_df_drop, columns=["City", "Country", "Lat", "Lon"])

#empty column to place hotel names
hotel_df["Hotel Name"]= ""

hotel_df

NameError: name 'conditions_df_drop' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
